[Pandas_ta](https://github.com/twopirllc/pandas-ta) kütüphanesini install ediyoruz

In [ ]:
!pip install pandas_ta

Gerekli kütüphaneleri içe aktarıyoruz.

In [3]:
import numpy as np
import yfinance as yf
import pandas as pd
import datetime as dt
import pandas_ta as ta

Yahoo Finance'teki hisse senedi isimlerini python list olarak değişkene atıyoruz. burada bist hisseleri için sonuna ".IS" eklemelisiniz. örneğin stocks_tr = ['AVOD.IS', 'ACSEL.IS', 'ADEL.IS', 'ADESE.IS', 'AFYON.IS']

In [10]:
stocks_us = ['AAPL', 'GOOG', 'MRNA', 'SHOP', 'AMC']
#stocks_tr = ['AVOD.IS', 'ACSEL.IS', 'ADEL.IS', 'ADESE.IS', 'AFYON.IS']

# tradingview'da yer alan populer Lazybear TTM Squeeze Mmentum Indikatörünün, ihtiyacım olan bölümünün pythona çevrilmiş hali
# alttaki yeşil ve kırmızı çubuklar sadece
def squeeze(data_df):
    length_KC = 20
    highest = data_df['High'].rolling(window = length_KC).max()
    lowest = data_df['Low'].rolling(window = length_KC).min()
    m1 = (highest + lowest)/2
    m_avg = data_df['Close'].rolling(window=length_KC).mean()
    data_df['value'] = (data_df['Close'] - (m1 + m_avg)/2)
    fit_y = np.array(range(0,length_KC))
    data_df['value'] = data_df['value'].rolling(window = length_KC).apply(lambda x:
                        np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
                        np.polyfit(fit_y, x, 1)[1], raw=True)
    return data_df

Her bir hisse senedi için verileri indiriyoruz ve işliyoruz. Squeeze indikatörünü yukarıda kendimiz yazdık. RSI, ADX ve Supertrend indikatörlerini pandas_ta kullanarak alıyoruz.

In [ ]:
for stock in stocks_us:
    df = yf.download(stock, group_by='Ticker', period = 'max', interval = '1d')
    df = df.reset_index()
    df.ta.rsi(length=6, append=True)
    df.ta.adx(append=True)
    df.ta.supertrend(length=10,multiplier=3,append=True)
    df=squeeze(df)
    df['stock'] = stock
    df.to_csv(f'ticker_{stock}.csv')
    print(df.tail(10))

Veriyi haftanın günlerine göre ayarlıyoruz. Piyasa cumartesi ve pazar kapalı olduğu için..


In [6]:
df.reset_index(inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
dfn = df.set_index('Date').resample('D').asfreq()
dfn['Day'] = dfn.index.strftime('%A')
cond = dfn['Day'].isin(['Saturday', 'Sunday'])
dfn = dfn[~cond].fillna(method='ffill').reset_index()

dfn

,Date,index,Open,High,Low,Close,Adj Close,Volume,RSI_6,ADX_14,DMP_14,DMN_14,SUPERT_10_3.0,SUPERTd_10_3.0,SUPERTl_10_3.0,SUPERTs_10_3.0,value,stock,Day
0,2013-12-18,0.0,19.180000,19.790001,18.900000,18.900000,13.593409,5457200.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,AMC,Wednesday
1,2013-12-19,1.0,18.900000,19.490000,18.809999,19.490000,14.017756,1605400.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,AMC,Thursday
2,2013-12-20,2.0,19.650000,19.770000,19.400000,19.680000,14.154408,737100.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,AMC,Friday
3,2013-12-23,3.0,19.790001,19.990000,19.500000,19.740000,14.197563,698100.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,AMC,Monday
4,2013-12-24,4.0,19.740000,19.820000,19.500000,19.530001,14.046528,213700.0,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN,NaN,AMC,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,2023-07-13,2406.0,4.430000,4.520000,4.390000,4.440000,4.440000,21391500.0,60.730129,14.062399,25.109163,18.339711,4.715902,-1.0,4.795612,4.715902,-0.218647,AMC,Thursday
2497,2023-07-14,2407.0,4.550000,4.570000,4.300000,4.330000,4.330000,17266400.0,47.090864,13.575946,22.801317,19.717802,4.715902,-1.0,4.795612,4.715902,-0.156500,AMC,Friday
2498,2023-07-17,2408.0,4.340000,4.390000,4.270000,4.370000,4.370000,12327200.0,51.813255,12.931079,21.840504,19.940394,4.715902,-1.0,4.795612,4.715902,-0.094957,AMC,Monday
2499,2023-07-18,2409.0,4.350000,4.540000,4.270000,4.310000,4.310000,18427600.0,44.641257,13.147287,24.964051,18.093009,4.715902,-1.0,4.795612,4.715902,-0.043461,AMC,Tuesday
